### Compare jpgs from nanfang and zhengzhou and get universal l and s values

In [1]:
import os
import cv2
import numpy as np
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed

from hls import scan_files, get_ls

In [4]:
def get_ls_cv2(image):
#     image = cv2.imread(image_name)

    # 图像归一化，且转换为浮点型
    hlsImg = image.astype(np.float32)
    hlsImg = hlsImg / 255.0
    # 颜色空间转换 BGR转为HLS
    hlsImg = cv2.cvtColor(hlsImg, cv2.COLOR_BGR2HLS)
    # 1.亮度
    l = np.average(hlsImg[:,:,1])
    # 2.饱和度
    s = np.average(hlsImg[:,:,2])

    return l, s

In [18]:
a = "/home/hdd0/Develop/tct/hls/nanfang/2017-09-07-08_47_59_19249_12678.jpg"
print(get_ls(a))

b = cv2.GaussianBlur(cv2.imread(a), ksize=(3,7), sigmaX=1.0, sigmaY=1.0)
# b = cv2.pyrDown(cv2.imread(a))
print(get_ls_cv2(b))

(0.91326296, 0.43797666)
(0.91334337, 0.38632008)


In [2]:
def collect_ls(jpg_dir):
    jpgs = [os.path.join(jpg_dir, f) for f in os.listdir(jpg_dir) if f.endswith(".jpg")]
    ls_list = []
    
    for jpg in jpgs:
        l, s = get_ls(jpg)
        ls_list.append([jpg, l, s])
    
    return ls_list

In [3]:
nanfang = "./nanfang"
zhengzhou = "./zhengzhou"

nanfang_ls = collect_ls(nanfang)
print(len(nanfang_ls))

zhengzhou_ls = collect_ls(zhengzhou)
print(len(zhengzhou_ls))

9571
2800


In [4]:
def percentile(values, percentages):
    length = len(percentages)
    percentages += [1.0]  # pad with one more value 
    mapping = [0 for i in range(length)]
    for value in values:
        for i in range(length):
            mapping[i] += 1 if percentages[i] < value < percentages[i+1] else 0
    return mapping

def percentile_(ls_list):
    percentages=[0.05 * i for i in range(1, 20)]
    l = [item[1] for item in ls_list]
    lp = percentile(l, percentages)
    s = [item[2] for item in ls_list]
    sp = percentile(s, percentages)
    for p, l_i, s_i in zip(percentages, lp, sp):
        print("> {:.2f}:  l {}  s {}".format(p, str(l_i).rjust(8), str(s_i).rjust(8)))

In [5]:
print("----nanfang----")
percentile_(nanfang_ls)

print("----zhengzhou----")
percentile_(zhengzhou_ls)

----nanfang----
> 0.05:  l        0  s      536
> 0.10:  l        0  s     1097
> 0.15:  l        0  s     1416
> 0.20:  l        0  s     1444
> 0.25:  l        0  s     1572
> 0.30:  l        0  s     1162
> 0.35:  l        0  s      775
> 0.40:  l        0  s      516
> 0.45:  l        3  s      437
> 0.50:  l        6  s      242
> 0.55:  l       30  s      135
> 0.60:  l      117  s       76
> 0.65:  l      416  s       69
> 0.70:  l      808  s       38
> 0.75:  l     1355  s       18
> 0.80:  l     1700  s        5
> 0.85:  l     2681  s        0
> 0.90:  l     2010  s        0
> 0.95:  l      445  s        0
----zhengzhou----
> 0.05:  l        0  s        4
> 0.10:  l        0  s      649
> 0.15:  l        0  s     1088
> 0.20:  l        0  s      549
> 0.25:  l        0  s      162
> 0.30:  l        0  s       34
> 0.35:  l        0  s       23
> 0.40:  l        1  s       13
> 0.45:  l        1  s       18
> 0.50:  l        5  s       25
> 0.55:  l        7  s       57
> 0.60

### align l and s, based on a threshold

In [7]:
def hls_trans_smart(image_name, depth, save_path, HLS_L=[0.9], HLS_S=[0.4, 0.5]):
    image = cv2.imread(image_name)

    # 图像归一化，且转换为浮点型
    hlsImg = image.astype(np.float32)
    hlsImg = hlsImg / 255.0
    # 颜色空间转换 BGR转为HLS
    hlsImg = cv2.cvtColor(hlsImg, cv2.COLOR_BGR2HLS)
    
    # 1.调整亮度
    l = np.average(hlsImg[:,:,1])
    i = len(HLS_L) - 1
    while i != -1 and HLS_L[i] > l:
        i -= 1
    if i != len(HLS_L)-1:
        hls_l = HLS_L[i+1]
        hlsImg[:, :, 1] = hls_l / l * hlsImg[:, :, 1]
        hlsImg[:, :, 1][hlsImg[:, :, 1] > 1] = 1
        print(image_name, "changing l", l, "to", hls_l)
        
    # 2.调整饱和度
    s = np.average(hlsImg[:,:,2])
    i = len(HLS_S) - 1
    while i != -1 and HLS_S[i] > s:
        i -= 1
    if i != len(HLS_S)-1:
        hls_s = HLS_S[i+1]
        hlsImg[:, :, 2] = hls_s / s * hlsImg[:, :, 2]
        hlsImg[:, :, 2][hlsImg[:, :, 2] > 1] = 1
        print(image_name, "changing s", s, "to", hls_s)
        
    # HLS2BGR
    hlsImg = cv2.cvtColor(hlsImg, cv2.COLOR_HLS2BGR)
    # 转换为8位unsigned char
    hlsImg = hlsImg * 255
    image = hlsImg.astype(np.uint8)
    
    tokens = image_name.rsplit(os.sep, depth+1)
    image_name_out = os.path.join(save_path, *tokens[1:])
    os.makedirs(os.path.dirname(image_name_out), exist_ok=True)

    cv2.imwrite(image_name_out, image)

In [8]:
def batch_hls_trans(image_names, depth, save_path):
    for image_name in image_names:
        hls_trans_smart(image_name, depth, save_path)

def process(image_path, depth, save_path):
    image_names = scan_files(image_path, postfix=".jpg")
    print("total of {} images to process".format(len(image_names)))

    executor = ProcessPoolExecutor(max_workers=cpu_count() - 2)
    tasks = []

    batch_size = 100
    for i in range(0, len(image_names), batch_size):
        batch = image_names[i : i+batch_size]
        tasks.append(executor.submit(batch_hls_trans, batch, depth, save_path))

    job_count = len(tasks)
    for future in as_completed(tasks):
        # result = future.result()  # get the returning result from calling fuction
        job_count -= 1
        # print("One Job Done, Remaining Job Count: {}".format(job_count))

In [9]:
# nanfang
nanfang = "./nanfang"
nanfang_ls = "./nanfang_ls"

process(nanfang, 0, nanfang_ls)

total of 9571 images to process
./nanfang/2017-09-07-12_13_59_27573_15087.jpg changing l 0.78364253 to 0.9
./nanfang/TC17027027_38158_51779.jpg changing l 0.866731 to 0.9
./nanfang/TC18014088_5726_14394.jpg changing s 0.052338872 to 0.4
./nanfang/2017-09-15-10_22_58_27427_14097.jpg changing l 0.87300485 to 0.9
./nanfang/2018-03-16-14_40_10_29563_41655.jpg changing l 0.65420353 to 0.9
./nanfang/TC18050519_11809_13716.jpg changing l 0.8843732 to 0.9
./nanfang/2018-06-01_12_40_20_9583_25463.jpg changing l 0.8917233 to 0.9
./nanfang/TC17027027_38158_51779.jpg changing s 0.4535033 to 0.5
./nanfang/2018-03-16-14_40_10_29563_41655.jpg changing s 0.22304602 to 0.4
./nanfang/2018-06-01_12_40_20_9583_25463.jpg changing s 0.31197104 to 0.4
./nanfang/TC18050519_11809_13716.jpg changing s 0.11561655 to 0.4
./nanfang/2017-09-07-12_13_59_27573_15087.jpg changing s 0.3818316 to 0.4
./nanfang/2017-09-19-18_57_18_27523_6632.jpg changing l 0.8005699 to 0.9
./nanfang/2018-01-15-15_49_03_25689_8363.jpg cha

./nanfang/TC18056497_8477_38397.jpg changing l 0.8483961 to 0.9
./nanfang/2017-09-19-17_59_45_38770_19070.jpg changing l 0.8186894 to 0.9
./nanfang/2017-09-07-16_26_22_47637_25276.jpg changing s 0.10739343 to 0.4
./nanfang/TC18056497_8477_38397.jpg changing s 0.3737878 to 0.4
./nanfang/2017-09-07-16_33_06_24083_41043.jpg changing s 0.28034052 to 0.4
./nanfang/TC18056497_41632_29370.jpg changing l 0.83235687 to 0.9
./nanfang/2017-09-19-19_41_31_37874_9335.jpg changing l 0.8874124 to 0.9
./nanfang/2017-09-19-17_59_45_38770_19070.jpg changing s 0.19797198 to 0.4
./nanfang/2017-09-19-19_30_11_40701_36100.jpg changing l 0.84005934 to 0.9
./nanfang/2017-10-09-17_24_09_30405_11056.jpg changing s 0.41589788 to 0.5
./nanfang/TC18056268_41636_30179.jpg changing s 0.22668467 to 0.4
./nanfang/TC18021597_32484_15354.jpg changing s 0.2100625 to 0.4
./nanfang/TC18056497_41632_29370.jpg changing s 0.4345192 to 0.5
./nanfang/TC17066064_40522_5491.jpg changing s 0.122793294 to 0.4
./nanfang/TC17014888_9

./nanfang/TC18014088_41189_13635.jpg changing s 0.07241982 to 0.4
./nanfang/2018-06-01_12_40_20_7476_27500.jpg changing l 0.89301103 to 0.9
./nanfang/2017-09-19-18_52_06_37704_7209.jpg changing s 0.4438528 to 0.5
./nanfang/TC18053245_9191_19988.jpg changing l 0.8341085 to 0.9
./nanfang/TC18053285_17461_28907.jpg changing l 0.860987 to 0.9
./nanfang/2018-06-01_12_40_20_34731_41315.jpg changing s 0.35532928 to 0.4
./nanfang/TC18053245_9191_19988.jpg changing s 0.12770537 to 0.4
./nanfang/2017-09-15-10_46_25_35930_25332.jpg changing l 0.81169546 to 0.9
./nanfang/2018-06-01_12_40_20_7476_27500.jpg changing s 0.31540096 to 0.4
./nanfang/TC18053285_17461_28907.jpg changing s 0.4941891 to 0.5
./nanfang/TC18024867_38350_43793.jpg changing l 0.87324196 to 0.9
./nanfang/TC18024867_38350_43793.jpg changing s 0.35877407 to 0.4
./nanfang/2017-10-09-18_21_10_32484_17380.jpg changing l 0.7521364 to 0.9
./nanfang/2017-09-19-18_08_49_15359_17174.jpg changing s 0.24083932 to 0.4
./nanfang/2017-09-15-10_

./nanfang/2017-09-07-16_33_06_48189_47557.jpg changing s 0.26617226 to 0.4
./nanfang/2018-06-01_12_40_20_43462_30341.jpg changing s 0.2743917 to 0.4
./nanfang/2018-05-05-16_48_27_20815_20328.jpg changing l 0.88599294 to 0.9
./nanfang/2018-05-05-16_48_27_20815_20328.jpg changing s 0.38928923 to 0.4
./nanfang/2018-06-01_12_40_20_10450_31011.jpg changing l 0.8970112 to 0.9
./nanfang/2018-06-01_12_40_20_10450_31011.jpg changing s 0.3212383 to 0.4
./nanfang/2017-09-19-19_18_05_41584_8016.jpg changing s 0.32921517 to 0.4
./nanfang/2017-10-11-15_21_37_48240_23467.jpg changing l 0.86062175 to 0.9
./nanfang/TC18055080_47457_27200.jpg changing s 0.17064781 to 0.4
./nanfang/2018-05-08-16_43_07_9737_40609.jpg changing s 0.23154105 to 0.4
./nanfang/TC18024596_42260_53792.jpg changing s 0.050120447 to 0.4
./nanfang/2018-01-15-17_45_12_8328_37864.jpg changing l 0.74686337 to 0.9
./nanfang/2018-05-08-16_43_07_31643_17004.jpg changing s 0.094142035 to 0.4
./nanfang/2017-09-15-10_30_47_38801_53153.jpg c

./nanfang/TC18053285_39742_49358.jpg changing l 0.885347 to 0.9
./nanfang/TC18056383_39481_22248.jpg changing l 0.88695765 to 0.9
./nanfang/2018-03-22-11_26_58_4897_15892.jpg changing s 0.20471752 to 0.4
./nanfang/2018-03-22-12_05_52_22001_39610.jpg changing l 0.696391 to 0.9
./nanfang/TC18056383_39481_22248.jpg changing s 0.16187112 to 0.4
./nanfang/2017-09-07-16_33_06_25838_21046.jpg changing l 0.77374274 to 0.9
./nanfang/TC18052925_27128_50694.jpg changing l 0.87333083 to 0.9
./nanfang/2017-09-07-18_05_49_39495_45034.jpg changing l 0.7759361 to 0.9
./nanfang/TC18052925_27128_50694.jpg changing s 0.4126422 to 0.5
./nanfang/2017-09-07-18_05_49_39495_45034.jpg changing s 0.44295615 to 0.5
./nanfang/2017-09-07-16_33_06_25838_21046.jpg changing s 0.28753436 to 0.4
./nanfang/2018-03-22-12_05_52_22001_39610.jpg changing s 0.16842441 to 0.4
./nanfang/2017-09-07-09_38_59_6415_32562.jpg changing l 0.8049942 to 0.9
./nanfang/TC17014226_11495_43549.jpg changing s 0.32301217 to 0.4
./nanfang/TC1

...

./nanfang/2018-03-22-15_17_54_23431_27849.jpg changing l 0.88681066 to 0.9
./nanfang/TC18050374_16401_8168.jpg changing l 0.814444 to 0.9
./nanfang/2018-03-22-15_17_54_23431_27849.jpg changing s 0.47427955 to 0.5
./nanfang/2017-09-15-10_28_29_23265_27860.jpg changing l 0.8582347 to 0.9
./nanfang/2018-03-22-15_12_14_42838_49397.jpg changing l 0.804702 to 0.9
./nanfang/2017-09-15-10_46_25_46487_40752.jpg changing s 0.23310919 to 0.4
./nanfang/2017-09-15-10_28_29_23265_27860.jpg changing s 0.31206596 to 0.4
./nanfang/TC18050374_16401_8168.jpg changing s 0.18443823 to 0.4
./nanfang/2018-03-22-15_12_14_42838_49397.jpg changing s 0.08216499 to 0.4
./nanfang/TC18026873_40216_33096.jpg changing l 0.78805476 to 0.9
./nanfang/2018-03-16-19_47_54_23048_32304.jpg changing l 0.8756827 to 0.9
./nanfang/TC18052588_31438_29844.jpg changing s 0.27674803 to 0.4
./nanfang/2018-03-16-19_47_54_23048_32304.jpg changing s 0.16521212 to 0.4
./nanfang/TC18026873_40216_33096.jpg changing s 0.1725076 to 0.4
./na

./nanfang/TC17020758_12800_29958.jpg changing l 0.88312393 to 0.9
./nanfang/2018-03-27-21_50_10_12745_13683.jpg changing l 0.7104995 to 0.9
./nanfang/2017-09-19-20_06_17_58026_29492.jpg changing s 0.15813884 to 0.4
./nanfang/TC17020758_12800_29958.jpg changing s 0.17403899 to 0.4
./nanfang/NJ18088753_27796_33250.jpg changing s 0.26409465 to 0.4
./nanfang/2018-03-27-21_50_10_12745_13683.jpg changing s 0.31018832 to 0.4
./nanfang/2018-01-15-17_36_38_40826_48087.jpg changing l 0.78866744 to 0.9
./nanfang/NJ18088152_16400_29415.jpg changing s 0.22084484 to 0.4
./nanfang/TC18014088_40650_6253.jpg changing s 0.051813994 to 0.4
./nanfang/2018-01-15-17_36_38_40826_48087.jpg changing s 0.36992198 to 0.4
./nanfang/2017-09-19-19_15_19_28568_34555.jpg changing l 0.73082346 to 0.9
./nanfang/2017-09-19-18_08_49_45858_20759.jpg changing s 0.18985717 to 0.4
./nanfang/2017-09-19-19_15_19_28568_34555.jpg changing s 0.236941 to 0.4
./nanfang/2017-09-07-08_50_02_17827_48024.jpg changing s 0.38578123 to 0.

./nanfang/2017-09-19-19_30_11_46558_12486.jpg changing l 0.8301196 to 0.9
./nanfang/2018-03-22-11_44_45_16613_44026.jpg changing s 0.20714171 to 0.4
./nanfang/2017-09-19-19_30_11_46558_12486.jpg changing s 0.4212657 to 0.5
./nanfang/TC17062962_12483_45052.jpg changing l 0.8439529 to 0.9
./nanfang/2017-09-07-17_52_27_45325_37242.jpg changing s 0.17143963 to 0.4
./nanfang/TC17015494_34433_44713.jpg changing s 0.2691589 to 0.4
./nanfang/TC17025476_26041_40307.jpg changing l 0.853553 to 0.9
./nanfang/2018-06-01_12_26_15_15211_20402.jpg changing l 0.8504908 to 0.9
./nanfang/2018-03-27-21_50_10_32793_34570.jpg changing l 0.7775921 to 0.9
./nanfang/2018-03-27-20_11_43_20991_29651.jpg changing l 0.7737516 to 0.9
./nanfang/TC17025476_26041_40307.jpg changing s 0.14955255 to 0.4
./nanfang/2018-06-01_12_26_15_15211_20402.jpg changing s 0.21412109 to 0.4
./nanfang/TC18051506_19666_28069.jpg changing l 0.854548 to 0.9
./nanfang/2018-03-27-20_11_43_20991_29651.jpg changing s 0.45427948 to 0.5
./nanf

./nanfang/TC18008562_48657_35502.jpg changing s 0.22480151 to 0.4
./nanfang/2018-06-20_18_28_33_7369_19913.jpg changing l 0.7933697 to 0.9
./nanfang/TC17021418_10214_35133.jpg changing l 0.8334754 to 0.9
./nanfang/2018-06-20_18_28_33_7369_19913.jpg changing s 0.037783947 to 0.4
./nanfang/TC17021418_10214_35133.jpg changing s 0.2636047 to 0.4
./nanfang/TC18052441_30555_21172.jpg changing l 0.8143292 to 0.9
./nanfang/2017-11-24-12_08_21_12006_26172.jpg changing l 0.80825496 to 0.9
./nanfang/TC18052441_30555_21172.jpg changing s 0.40556434 to 0.5
./nanfang/2018-06-20_18_52_15_35831_10299.jpg changing l 0.81861186 to 0.9
./nanfang/2018-06-20_18_52_15_35831_10299.jpg changing s 0.1856398 to 0.4
./nanfang/2017-11-24-12_08_21_12006_26172.jpg changing s 0.21159185 to 0.4
./nanfang/2018-01-15-17_54_47_5655_13802.jpg changing l 0.7612103 to 0.9
./nanfang/2018-01-15-17_54_47_5655_13802.jpg changing s 0.24055219 to 0.4
./nanfang/2017-09-19-19_18_05_52413_18578.jpg changing l 0.8119791 to 0.9
./nan

./nanfang/TC18056268_12998_33798.jpg changing s 0.22262931 to 0.4
./nanfang/TC17027027_25963_18779.jpg changing l 0.8810469 to 0.9
./nanfang/2018-04-19-21_58_49_4026_16172.jpg changing s 0.13597606 to 0.4
./nanfang/TC17027027_25963_18779.jpg changing s 0.40302303 to 0.5
./nanfang/2017-09-19-19_23_06_25173_25398.jpg changing l 0.8671641 to 0.9
./nanfang/2017-09-19-19_23_06_25173_25398.jpg changing s 0.38692823 to 0.4
./nanfang/2017-11-24-13_14_56_37305_5373.jpg changing l 0.81675977 to 0.9
./nanfang/2017-11-24-13_14_56_37305_5373.jpg changing s 0.18742122 to 0.4
./nanfang/2018-01-15-15_32_42_22094_23917.jpg changing l 0.7917422 to 0.9
./nanfang/2018-01-15-15_32_42_22094_23917.jpg changing s 0.14167449 to 0.4
./nanfang/TC18008562_9567_28215.jpg changing l 0.8991501 to 0.9
./nanfang/TC18008562_9567_28215.jpg changing s 0.24036513 to 0.4
./nanfang/2017-09-07-09_43_37_46289_7861.jpg changing l 0.80880934 to 0.9
./nanfang/TC17011948_43492_42977.jpg changing l 0.83767027 to 0.9
./nanfang/2018

In [10]:
# zhengzhou
zhengzhou = "./zhengzhou"
zhengzhou_ls = "./zhengzhou_ls"

process(zhengzhou, 0, zhengzhou_ls)

total of 2800 images to process
./zhengzhou/L18-38597_57840_28530.jpg changing s 0.19705068 to 0.4
./zhengzhou/L18-37404_42643_33391.jpg changing l 0.7383107 to 0.9
./zhengzhou/L18-36835_36569_65024.jpg changing l 0.8060365 to 0.9
./zhengzhou/L18-11340_34760_29746.jpg changing l 0.86717534 to 0.9
./zhengzhou/L18-38394_57247_58925.jpg changing l 0.80370957 to 0.9
./zhengzhou/L18-14695_35359_42813.jpg changing l 0.757703 to 0.9
./zhengzhou/L18-18094_32918_28225.jpg changing l 0.7154411 to 0.9
./zhengzhou/L18-11340_34760_29746.jpg changing s 0.24164127 to 0.4
./zhengzhou/L18-37404_42643_33391.jpg changing s 0.113485985 to 0.4
./zhengzhou/L18-36835_36569_65024.jpg changing s 0.14162755 to 0.4
./zhengzhou/L18-38394_57247_58925.jpg changing s 0.1823014 to 0.4
./zhengzhou/L18-14695_35359_42813.jpg changing s 0.15622298 to 0.4
./zhengzhou/L18-18094_32918_28225.jpg changing s 0.16155888 to 0.4
./zhengzhou/L18-36874_37161_30943.jpg changing l 0.7881583 to 0.9
./zhengzhou/L18-18151_30487_40980.jp

./zhengzhou/L18-11340_32328_44338.jpg changing l 0.8825376 to 0.9
./zhengzhou/L18-11381_62700_58911.jpg changing s 0.13608958 to 0.4
./zhengzhou/L18-12004_48730_41598.jpg changing l 0.84083986 to 0.9
./zhengzhou/L18-11340_32328_44338.jpg changing s 0.22026668 to 0.4
./zhengzhou/L18-36835_62105_48000.jpg changing l 0.81300724 to 0.9
./zhengzhou/L18-11350_49948_39467.jpg changing l 0.7123144 to 0.9
./zhengzhou/L18-35910_43246_58910.jpg changing s 0.16338918 to 0.4
./zhengzhou/L18-11350_49948_39467.jpg changing s 0.16431718 to 0.4
./zhengzhou/L18-36835_62105_48000.jpg changing s 0.13075288 to 0.4
./zhengzhou/L18-11350_26844_57707.jpg changing l 0.78774685 to 0.9
./zhengzhou/L18-12004_48730_41598.jpg changing s 0.2190535 to 0.4
./zhengzhou/L18-38394_36575_44333.jpg changing l 0.77027965 to 0.9
./zhengzhou/L18-11337_29275_27310.jpg changing l 0.81468534 to 0.9
./zhengzhou/L18-11350_26844_57707.jpg changing s 0.1490058 to 0.4
./zhengzhou/L18-11337_29275_27310.jpg changing s 0.19175199 to 0.4

./zhengzhou/L18-18151_36567_49492.jpg changing l 0.7965686 to 0.9
./zhengzhou/L18-38213_55404_34903.jpg changing l 0.8449836 to 0.9
./zhengzhou/L18-11337_30491_33390.jpg changing l 0.80893445 to 0.9
./zhengzhou/L18-11376_53597_44334.jpg changing l 0.7757384 to 0.9
./zhengzhou/L18-18151_36567_49492.jpg changing s 0.15773314 to 0.4
./zhengzhou/L18-11376_53597_44334.jpg changing s 0.13134179 to 0.4
./zhengzhou/L18-11337_30491_33390.jpg changing s 0.18879446 to 0.4
./zhengzhou/L18-36835_28057_48000.jpg changing l 0.8166377 to 0.9
./zhengzhou/L18-38213_55404_34903.jpg changing s 0.19658338 to 0.4
./zhengzhou/L18-05845_57836_51310.jpg changing l 0.768477 to 0.9
./zhengzhou/L18-11337_54811_51630.jpg changing l 0.7842915 to 0.9
./zhengzhou/L18-36835_28057_48000.jpg changing s 0.13436615 to 0.4
./zhengzhou/L18-11340_42056_40690.jpg changing l 0.8808631 to 0.9
./zhengzhou/L18-11340_42056_40690.jpg changing s 0.2062296 to 0.4
./zhengzhou/L18-05845_57836_51310.jpg changing s 0.11069458 to 0.4
./zh

./zhengzhou/L18-05845_57836_59822.jpg changing s 0.11808484 to 0.4
./zhengzhou/L18-14695_34143_59837.jpg changing s 0.19861038 to 0.4
./zhengzhou/L18-18094_59670_61057.jpg changing l 0.7432186 to 0.9
./zhengzhou/L18-18094_59670_61057.jpg changing s 0.1242562 to 0.4
./zhengzhou/L18-12004_59674_47678.jpg changing l 0.82859874 to 0.9
./zhengzhou/L18-11340_46920_39474.jpg changing l 0.8628928 to 0.9
./zhengzhou/L18-00050_31701_56185.jpg changing s 0.21226677 to 0.4
./zhengzhou/L18-12004_58458_51326.jpg changing l 0.8375856 to 0.9
./zhengzhou/L18-35817_51161_27312.jpg changing l 0.840987 to 0.9
./zhengzhou/L18-12004_59674_47678.jpg changing s 0.21349102 to 0.4
./zhengzhou/L18-11340_46920_39474.jpg changing s 0.23581292 to 0.4
./zhengzhou/L18-05845_52972_53742.jpg changing l 0.7186287 to 0.9
./zhengzhou/L18-35817_51161_27312.jpg changing s 0.16321065 to 0.4
./zhengzhou/L18-05845_52972_53742.jpg changing s 0.14210524 to 0.4
./zhengzhou/L18-11359_32301_27912.jpg changing l 0.7716729 to 0.9
./z

./zhengzhou/L18-38594_51155_52232.jpg changing l 0.7549323 to 0.9
./zhengzhou/L18-18151_41431_51924.jpg changing l 0.7831272 to 0.9
./zhengzhou/L18-11376_30493_38254.jpg changing l 0.7195934 to 0.9
./zhengzhou/L18-18151_41431_51924.jpg changing s 0.16603853 to 0.4
./zhengzhou/L18-38594_51155_52232.jpg changing s 0.18548739 to 0.4
./zhengzhou/L18-11376_30493_38254.jpg changing s 0.22533917 to 0.4
./zhengzhou/L18-05845_63916_59822.jpg changing l 0.76110685 to 0.9
./zhengzhou/L18-11381_62700_41887.jpg changing l 0.76620543 to 0.9
./zhengzhou/L18-11404_36574_32778.jpg changing l 0.6282799 to 0.9
./zhengzhou/L18-11340_28680_37042.jpg changing l 0.8745019 to 0.9
./zhengzhou/L18-05845_63916_59822.jpg changing s 0.11079326 to 0.4
./zhengzhou/L18-38394_48735_61357.jpg changing l 0.7926021 to 0.9
./zhengzhou/L18-11381_62700_41887.jpg changing s 0.1238282 to 0.4
./zhengzhou/L18-11404_36574_32778.jpg changing s 0.26131773 to 0.4
./zhengzhou/L18-38394_48735_61357.jpg changing s 0.19789018 to 0.4
./

...

./zhengzhou/L18-00050_59669_27001.jpg changing s 0.35427412 to 0.4
./zhengzhou/L18-18151_57239_36116.jpg changing l 0.7965559 to 0.9
./zhengzhou/L18-38213_52972_56791.jpg changing s 0.21508041 to 0.4
./zhengzhou/L18-35817_52377_29744.jpg changing s 0.16160063 to 0.4
./zhengzhou/L18-11359_33517_35208.jpg changing s 0.15101789 to 0.4
./zhengzhou/L18-18151_57239_36116.jpg changing s 0.15514478 to 0.4
./zhengzhou/L18-12004_48730_57406.jpg changing s 0.19479622 to 0.4
./zhengzhou/L18-11350_53596_33387.jpg changing l 0.7822897 to 0.9
./zhengzhou/L18-18139_54210_36424.jpg changing s 0.33173987 to 0.4
./zhengzhou/L18-11404_26846_31562.jpg changing l 0.6940485 to 0.9
./zhengzhou/L18-36835_60889_60160.jpg changing l 0.82595485 to 0.9
./zhengzhou/L18-11350_53596_33387.jpg changing s 0.12690099 to 0.4
./zhengzhou/L18-38597_32304_62578.jpg changing s 0.22769915 to 0.4
./zhengzhou/L18-11337_51163_27310.jpg changing l 0.782631 to 0.9
./zhengzhou/L18-11337_51163_27310.jpg changing s 0.21599977 to 0.4


./zhengzhou/L18-11381_31084_34591.jpg changing s 0.1480389 to 0.4
./zhengzhou/L18-18130_49946_54063.jpg changing s 0.12585105 to 0.4
./zhengzhou/L18-11344_27443_27913.jpg changing l 0.6618512 to 0.9
./zhengzhou/L18-11404_43870_47370.jpg changing s 0.15011297 to 0.4
./zhengzhou/L18-11344_27443_27913.jpg changing s 0.22071812 to 0.4
./zhengzhou/L18-37404_57235_27311.jpg changing l 0.7515251 to 0.9
./zhengzhou/L18-14695_29279_37949.jpg changing l 0.7554435 to 0.9
./zhengzhou/L18-11404_28062_30346.jpg changing l 0.76925534 to 0.9
./zhengzhou/L18-14695_29279_37949.jpg changing s 0.16219167 to 0.4
./zhengzhou/L18-37404_57235_27311.jpg changing s 0.10349523 to 0.4
./zhengzhou/L18-38597_52976_28530.jpg changing s 0.2077488 to 0.4
./zhengzhou/L18-38594_62099_32776.jpg changing l 0.80393976 to 0.9
./zhengzhou/L18-11404_28062_30346.jpg changing s 0.1709924 to 0.4
./zhengzhou/L18-36835_28057_46784.jpg changing l 0.7946233 to 0.9
./zhengzhou/L18-35910_42030_57694.jpg changing l 0.6751826 to 0.9
./z

./zhengzhou/L18-11344_40819_46153.jpg changing l 0.65541065 to 0.9
./zhengzhou/L18-05845_34732_37934.jpg changing l 0.6792267 to 0.9
./zhengzhou/L18-38594_29267_31560.jpg changing s 0.19939849 to 0.4
./zhengzhou/L18-11381_32300_51615.jpg changing l 0.7644192 to 0.9
./zhengzhou/L18-05845_34732_37934.jpg changing s 0.18044534 to 0.4
./zhengzhou/L18-35817_29273_51632.jpg changing s 0.16433111 to 0.4
./zhengzhou/L18-11344_40819_46153.jpg changing s 0.22111435 to 0.4
./zhengzhou/L18-38594_28051_30344.jpg changing l 0.816495 to 0.9
./zhengzhou/L18-11381_32300_51615.jpg changing s 0.15013382 to 0.4
./zhengzhou/L18-38594_28051_30344.jpg changing s 0.17293262 to 0.4
./zhengzhou/L18-10193_48720_55270.jpg changing l 0.85051394 to 0.9
./zhengzhou/L18-36835_34137_33408.jpg changing l 0.79476076 to 0.9
./zhengzhou/L18-05845_59052_28206.jpg changing l 0.7601971 to 0.9
./zhengzhou/L18-36835_34137_33408.jpg changing s 0.15680456 to 0.4
./zhengzhou/L18-11404_40222_46154.jpg changing l 0.7337672 to 0.9
.

./zhengzhou/L18-12004_57242_52542.jpg changing l 0.86300844 to 0.9
./zhengzhou/L18-37404_52371_28527.jpg changing l 0.75278395 to 0.9
./zhengzhou/L18-18139_42050_30344.jpg changing s 0.38622403 to 0.4
./zhengzhou/L18-11340_51784_57714.jpg changing l 0.8641949 to 0.9
./zhengzhou/L18-12004_57242_52542.jpg changing s 0.17442131 to 0.4
./zhengzhou/L18-18126_42635_54066.jpg changing l 0.8887826 to 0.9
./zhengzhou/L18-11340_51784_57714.jpg changing s 0.23942399 to 0.4
./zhengzhou/L18-18130_37786_50415.jpg changing l 0.6886349 to 0.9
./zhengzhou/L18-37404_52371_28527.jpg changing s 0.10317243 to 0.4
./zhengzhou/L18-12004_35354_57406.jpg changing l 0.85834813 to 0.9
./zhengzhou/L18-11337_28059_37038.jpg changing l 0.7321161 to 0.9
./zhengzhou/L18-12004_35354_57406.jpg changing s 0.21004103 to 0.4
./zhengzhou/L18-18126_42635_54066.jpg changing s 0.25086537 to 0.4
./zhengzhou/L18-18130_37786_50415.jpg changing s 0.16990803 to 0.4
./zhengzhou/L18-38213_50540_43415.jpg changing l 0.83670324 to 0.9

./zhengzhou/L18-11376_62109_52846.jpg changing s 0.17477284 to 0.4
./zhengzhou/L18-11350_59676_56491.jpg changing l 0.7531276 to 0.9
./zhengzhou/L18-11350_59676_56491.jpg changing s 0.14289306 to 0.4
./zhengzhou/L18-11404_53598_44938.jpg changing l 0.75544447 to 0.9
./zhengzhou/L18-11404_53598_44938.jpg changing s 0.16611429 to 0.4
./zhengzhou/L18-11340_29896_27314.jpg changing l 0.82879317 to 0.9
./zhengzhou/L18-11340_29896_27314.jpg changing s 0.2736761 to 0.4
./zhengzhou/L18-36874_54185_26079.jpg changing l 0.7905306 to 0.9
./zhengzhou/L18-36874_54185_26079.jpg changing s 0.18183486 to 0.4
./zhengzhou/L18-38594_59667_36424.jpg changing l 0.8153388 to 0.9
./zhengzhou/L18-38594_59667_36424.jpg changing s 0.166052 to 0.4


In [48]:
from PIL import Image
import cv2
import numpy as np

def hls_trans_smart(image, HLS_L=[0.9], HLS_S=[0.4, 0.5]):
    # image = cv2.imread(image_name)
    image = np.asarray(image)

    # 图像归一化，且转换为浮点型
    hlsImg = image.astype(np.float32)
    hlsImg = hlsImg / 255.0
    # 颜色空间转换 BGR转为HLS
    hlsImg = cv2.cvtColor(hlsImg, cv2.COLOR_BGR2HLS)
    
    # 1.调整亮度
    l = np.average(hlsImg[:,:,1])
    i = len(HLS_L) - 1
    while i != -1 and HLS_L[i] > l:
        i -= 1
    if i != len(HLS_L)-1:
        hls_l = HLS_L[i+1]
        hlsImg[:, :, 1] = hls_l / l * hlsImg[:, :, 1]
        hlsImg[:, :, 1][hlsImg[:, :, 1] > 1] = 1
#         print(image_name, "changing l", l, "to", hls_l)
        
    # 2.调整饱和度
    s = np.average(hlsImg[:,:,2])
    i = len(HLS_S) - 1
    while i != -1 and HLS_S[i] > s:
        i -= 1
    if i != len(HLS_S)-1:
        hls_s = HLS_S[i+1]
        hlsImg[:, :, 2] = hls_s / s * hlsImg[:, :, 2]
        hlsImg[:, :, 2][hlsImg[:, :, 2] > 1] = 1
#         print(image_name, "changing s", s, "to", hls_s)
        
    # HLS2BGR
    hlsImg = cv2.cvtColor(hlsImg, cv2.COLOR_HLS2BGR)
    # 转换为8位unsigned char
    hlsImg = hlsImg * 255
    image = hlsImg.astype(np.uint8)
    
    return Image.fromarray(image)

In [49]:
a = "/home/hdd0/Develop/tct/hls/aaa.jpg"

img = Image.open(a)
img_hls = hls_trans_smart(img)
img_hls.save("./ddd.jpg")

In [39]:
import cv2

a = "/home/hdd0/Develop/tct/hls/zhengzhou/L18-00050_36565_59833.jpg"
img = cv2.imread(a)



img[:,:][img[:,:] > 220] = 255
img = cv2.medianBlur(img, ksize=5)


# img = cv2.medianBlur(img, ksize=3)
# img = cv2.GaussianBlur(img, ksize=(7,7), sigmaX=0.5, sigmaY=0.5)
# img = cv2.GaussianBlur(img, ksize=(5,5), sigmaX=0.5, sigmaY=0.5)
# img = cv2.GaussianBlur(img, ksize=(5,5), sigmaX=0.5, sigmaY=0.5)

cv2.imwrite("./aaa.jpg", img)

True

In [44]:
a = "/home/hdd0/Develop/tct/hls/zhengzhou/L18-00050_36565_59833.jpg"
img = cv2.imread(a)

img = cv2.pyrDown(img)

cv2.imwrite("./ccc.jpg", img)

True